<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Numerical Determination of Eigenenergies for an Asymmetric Potential
## Examples - Quantum Mechanics
<section class="post-meta">
By Henning Goa Hugdal, Øystein Hiåsen and Peter Berg
</section>
Last edited: October 31st 2018
___

В блокноте про [*Численное определение собственных энергий гармонического осциллятора*](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/numerical_determination_eigenvalues_harmonic_oscillator.ipynb), мы показали, как численно определить собственные энергии потенциала гармонического осциллятора. Используя тот факт, что потенциал был симметричным, и, следовательно, волновые функции были симметричными или антисимметричными, мы могли легко выбрать правильные граничные условия для интересующей нас волновой функции. Однако для асимметричного потенциала у нас нет такой информации! Все, что мы знаем, это то, что собственные функции должны быть интегрируемыми и гладкими. И этого на самом деле достаточно, чтобы решить эту проблему.

Рассмотрим следующий потенциал:
$ $ V(x) = ax^4-b(x+c)^2+d, $ $
с $a=1$, $b=1,5$, $c=0,2$ и $d=1,17$. Потенциал немного асимметричен, как видно на графике ниже.

In [ ]:
%matplotlib inline
#from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

# Set common figure parameters:
newparams = {'axes.labelsize': 14, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'figure.subplot.wspace': 0.4,
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': False, 
             'legend.handlelength': 1.5}
plt.rcParams.update(newparams)

In [ ]:
n = 1000    # number of points per unit on x-axis
dx = 1.0/n
p = 10      # which x-values to include
linP = np.linspace(0,p*n-1, p*n, True)
linM = np.linspace(-(p*n-1), 0, p*n, True)
xP = linP/n # x-values in positive direction
xM = linM/n # x-values in negative direction

a = 1.0
b = 1.5
c = 0.2
d = 1.17
VP = a*xP**4 - b*(xP+c)**2 + d # potential for x>0
VM = a*xM**4 - b*(xM+c)**2 + d # potential for x<0

plt.figure()
plt.plot(xM, VM, 'r', xP, VP, 'r')
plt.grid()
plt.ylim([-1, 10])
plt.xlim([-2, 2])
plt.title(r'The Potential Under Consideration')
plt.xlabel(r'$x$')
plt.ylabel(r'$V(x)$');

Установка $\hbar = 1$, $m = 1$ для простоты уравнение Шредингера гласит
$$\left[-\frac{1}{2}\frac{\rm d^2}{{\rm d}x^2} +  V(x) \right]\psi(x) = E \psi(x), $$ 
получаем следующее уравнение для $\psi"(x)$,
$$\psi''(x) = 2[V(x)-E]\psi(x).$$
Дискретизируя ось $x$ и волновую функцию и используя метод центральной разности второго порядка, мы получаем формулу для значения функции $\psi_{i+1}$ на основе двух предыдущих точек,
$$\psi_{i+1} = 2\psi_i-\psi_{i-1}-2(\Delta x)^2\left[E-V(x) \right]\psi_i, $$
где $\Delta x$ - расстояние между двумя точками $x_i$ и $x_{i+1}$ на оси $x$. Для первого значения функции с каждой стороны источника мы используем
$$ \psi_1 = \psi_0 + \psi_0'\Delta x,$$
где $\psi_0$ и $\psi_0'$ - начальное значение в начале координат для волновой функции и наклон волновой функции соответственно.

Как уже упоминалось, мы должны найти функции, которые являются интегрируемыми и гладкими. Поэтому мы попробуем выполнить следующую процедуру:
Для данной энергии используйте приведенные выше уравнения, чтобы найти волновые функции как для положительных, так и для отрицательных значений $x$, которые квадратично интегрируемы. Для этого нам нужно определить функции, которые находят начальное значение наклона таким образом, чтобы волновые функции приближались к нулю как $x \rightarrow \infty$, например, с помощью метода деления пополам. Важно отметить, что критерий для метода деления пополам будет варьироваться в зависимости от того, приближается ли волновая функция к нулю сверху или снизу. Следовательно, нам нужны две разные функции, и для простоты мы также определили отдельные функции для положительных и отрицательных значений $x$. Эти четыре функции определены ниже.

In [ ]:
def findSlopeP(Ein, acc):
    """Function for positive x, approaches zero from above.
    Takes energy and desired accuracy as imput. """
    S1 = -30.0    # lower limit for slope
    S2 = 30.0     # upper limit for slope
    DeltaSP = 1.0 
    S = 0         # starting value for slope
    while DeltaSP > acc:
        for i in linP[0:-1]:
            if i == 0:
                f1[i+1] = f1[i] + dx*S
            else:
                f1[i+1] = -f1[i-1] + f1[i]*(2-dx**2*2*(Ein-VP[i]))
            if f1[i] < -20:
                # the wave function shoots off towards minus infinity, adjust slope
                S1 = S
                S = S1 + (S2-S1)/2 
                break
            elif f1[i] > 20:
                # the wave function shoots off towards plus infinity, adjust slope
                S2 = S
                S = S2 - (S2-S1)/2
                break
        DeltaSP = S2-S1 # if DeltaSP is smaller than the given accuracy, the function returns the calculated slope
    return S


def findSlopeM(Ein, acc):
    """Function for negative x, approaches zero from above. """
    S1 = -30.0
    S2 = 30.0
    DeltaSM = 1.0
    S = 0
    while DeltaSM > acc:
        for i in linP[1:-1]:
            if i == 1:
                f2[-(i+1)] = f2[-i] + dx*S
            else:
                f2[-(i+1)] = -f2[-(i-1)] + f2[-i]*(2-dx**2*2*(Ein-VM[-i]))
            if f2[-i] <- 20:
                S1 = S
                S = S1 + (S2-S1)/2
                break
            elif f2[-i] > 20:
                S2 = S
                S = S2 - (S2-S1)/2
                break
        DeltaSM = (S2-S1)
    return S


def findSlopeP2(Ein, acc):
    """Function for positive x, approaches zero from below. """
    S1 = -30.0
    S2 = 30.0
    DeltaSP = 1.0
    S = 0.0
    while DeltaSP > acc:
        for i in linP[0:-1]:
            if i == 0:
                f1[i+1] = f1[i] + dx*S
            else:
                f1[i+1] = -f1[i-1] + f1[i]*(2-dx**2*2*(Ein-VP[i]))
            if f1[i] > 20:
                S1 = S
                S = S1 + (S2-S1)/2
                break
            elif f1[i] < -20:
                S2 = S
                S = S2 - (S2-S1)/2
                break
        DeltaSP = abs(S2-S1)
    return S


def findSlopeM2(Ein, acc):
    """Function for negative x, approaches zero from below. """
    S1 = -30.0
    S2 = 30.0
    DeltaSM = 1.0
    S = 0.0
    while DeltaSM > acc:
        for i in linP[1:-1]:
            if i == 1:
                f2[-(i+1)] = f2[-i] + dx*S
            else:
                f2[-(i+1)] = -f2[-(i-1)] + f2[-i]*(2-dx**2*2*(Ein-VM[-i]))
            if f2[-i] > 20:
                S1 = S
                S = S1 + (S2-S1)/2
                break
            elif f2[-i] < -20:
                S2 = S
                S = S2 - (S2-S1)/2
                break
        DeltaSM = abs(S2-S1)
    return S

Теперь нам нужно найти значения энергии, которые дают одинаковый наклон в обоих направлениях. Используя тот факт, что возбужденное состояние $n^{\rm th}$ имеет узлы $n-1$, мы можем определить, какие комбинации вышеперечисленных функций использовать. Теперь все готово к тому, чтобы начать вычислять собственные энергии. Мы начинаем с энергии основного состояния, $E_0$.

### Основное Состояние

Основное состояние имеет нулевые узлы, поэтому мы будем использовать функции, в которых волновая функция приближается к нулю сверху, __findSlopeP__ и __findSlopeM__, начиная поиск в интервале $0<E<2$ с длиной шага 0.1. Построение наклона как функции энергии как для положительных, так и для отрицательных значений $x$ покажет, дает ли какое-либо из значений энергии гладкую волновую функцию.

In [ ]:
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0

acc = 0.01
N = 2
E = np.linspace(0,N,10*N+1,True)
lin = np.linspace(0,10*N,10*N+1,True)
SP = np.zeros(10*N+1)
SM = np.zeros(10*N+1)
for k in lin:
    SP[k] = findSlopeP(E[k], acc)
    SM[k] = findSlopeM(E[k], acc)

plt.figure()
plt.plot(E, SP, 'b', E, -SM, 'g')
plt.xticks(np.arange(min(E), max(E), (max(E)-min(E))/10))
plt.grid()
plt.xlabel('Energy')
plt.ylabel('Slope');

Мы видим, что существует пересечение для энергии $E_0 \approx 1.1$. После некоторых проб и ошибок мы видим, что энергия основного состояния составляет приблизительно $E_0 = 1.09$, что дает довольно плавную функцию, как показано ниже. Так что, похоже, этот метод работает!

In [ ]:
E0 = 1.09
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0
acc = 1e-6
SP0 = findSlopeP(E0, acc)
SM0 = findSlopeM(E0, acc)

print('Right slope: %s' % SP0)
print('Left slope: %s' % -SM0)


# Plot ground state
plt.figure()
plt.plot(xP, f1, 'b', xM, f2, 'b')
plt.xlim([-3,3])
plt.ylim([-0.5,2])
plt.ylabel('$\psi(x)$')
plt.xlabel('$x$')
plt.title('Ground State')
plt.grid();

Теперь мы видим, что вычисленные наклоны очень похожи. Чтобы получить более высокую точность, мы могли бы использовать метод деления пополам для энергий, близких к $1.09$.

### Первое Возбужденное Состояние

Для первого возбужденного состояния теорема об узле гласит, что у нас будет один узел. Следовательно, волновая функция будет приближаться к оси $x$ снизу с одной стороны. Но с какой стороны от источника будет находиться узел? Глядя на график потенциала выше, мы видим, что потенциал ниже с правой стороны, и, следовательно, эта область более "разрешена". Следовательно, узел, вероятно, будет находиться справа от источника. Чтобы найти $E_1$, мы затем должны использовать вторую функцию __findSlopeP2__ для положительных значений $x$.

In [ ]:
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0

acc = 0.01
N = 2
start = 1
E = np.linspace(start, N+start, 10*N+1)
lin = np.linspace(0, 10*N, 10*N+1)
SP = np.zeros(10*N+1)
SM = np.zeros(10*N+1)
for k in lin:
    SP[k] = findSlopeP2(E[k], acc)
    SM[k] = findSlopeM(E[k], acc)
    
    
plt.figure()
plt.plot(E, SP, 'b', E, -SM, 'g')
plt.xticks(np.arange(min(E), max(E), (max(E)-min(E))/10))
plt.grid()
plt.xlabel('Energy')
plt.ylabel('Slope');

Здесь мы видим, что существует пересечение в $E_1 \approx 2.3$, или, точнее, $E_1 = 2.33$.

In [ ]:
E1 = 2.33
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0
acc = 1e-6
SP1 = findSlopeP2(E1, acc)
SM1 = findSlopeM(E1, acc)

print('Right slope: %s' % SP1)
print('Left slope: %s' % -SM1)

# Plot first excited state
plt.figure()
plt.plot(xP, f1, 'b', xM, f2, 'b')
plt.xlim([-3,3])
plt.ylim([-2,2.5])
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.title('First Excited State')
plt.grid();

### Второе Возбужденное Состояние

Для второго возбужденного состояния мы ожидаем два узла, но где? Вероятно, они будут по обе стороны от начала координат, но сдвинуты вправо по сравнению с симметричным потенциалом. Следовательно, волновая функция будет приближаться к оси $x$ снизу с обеих сторон, что делает второй набор функций __findSlopeP2__ и __findSlopeM2__ необходимым как для отрицательных, так и для положительных значений $x$.

In [ ]:
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0

acc = 0.01
N = 3
start = 2
E = np.linspace(start, N+start, 10*N+1)
lin = np.linspace(0, 10*N, 10*N+1)
SP = np.zeros(10*N+1)
SM = np.zeros(10*N+1)

for k in lin:
    SP[k] = findSlopeP2(E[k], acc)
    SM[k] = findSlopeM2(E[k], acc)
    
    
plt.figure()
plt.plot(E,(SP),'b',E,-(SM),'g')
plt.xticks(np.arange(min(E), max(E), (max(E)-min(E))/10))
plt.grid()
plt.xlabel('Energy')
plt.ylabel('Slope');

Пересечение находится на уровне $E_2 \approx 4.2$, и дальнейшее исследование дает более точное значение $E_2 = 4.21$. 

In [ ]:
E2 = 4.21
f1 = np.zeros(p*n)
f1[0] = 1.0
f2 = np.zeros(p*n)
f2[-1] = 1.0
acc = 1e-6
SP2 = findSlopeP2(E2, acc)
SM2 = findSlopeM2(E2, acc)

print('Right slope: %s' % SP2)
print('Left slope: %s' % -SM2)

# Plot first excited state
plt.figure()
plt.plot(xP, f1, 'b', xM, f2, 'b')
plt.xlim([-3,3])
plt.ylim([-2,2.5])
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.title('Second Excited State')
plt.grid();

Из приведенного выше графика мы видим, что предположение относительно узлов было правильным. Продолжая в том же духе, мы могли бы определить энергии и построить волновые функции для более высоких состояний. Итак, мы показали, что на самом деле можно найти собственные энергии даже для асимметричного потенциала.